<a href="https://colab.research.google.com/github/francescopandolfo/TESI_LM32/blob/main/PCA%20-%20variazione_relativa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from sklearn.utils import shuffle

import warnings
warnings.filterwarnings("ignore")
pathDrive = 'drive/MyDrive/UNICT/_TESI/dati/'

df = pd.read_csv(pathDrive + 'dati_interpolati_diff_1d-2d_8gg.csv', sep=';')

df['edam_ecne_eplu_diff'] = df['edam_ecne_eplu_diff'].str.replace(',','.').astype(float)#
df['edam_ecor_emcn_diff'] = df['edam_ecor_emcn_diff'].str.replace(',','.').astype(float)
df['edam_emsg_eplu_diff'] = df['edam_emsg_eplu_diff'].str.replace(',','.').astype(float)
df['edam_epdn_ecne_diff'] = df['edam_epdn_ecne_diff'].str.replace(',','.').astype(float)
df['edam_epdn_emcn_diff'] = df['edam_epdn_emcn_diff'].str.replace(',','.').astype(float)
df['eint_ecpn_emgl_diff'] = df['eint_ecpn_emgl_diff'].str.replace(',','.').astype(float)#
df['eint_esln_emgl_diff'] = df['eint_esln_emgl_diff'].str.replace(',','.').astype(float)
df['eint_esln_espc_diff'] = df['eint_esln_espc_diff'].str.replace(',','.').astype(float)
df['emeg_ecpn_emgl_diff'] = df['emeg_ecpn_emgl_diff'].str.replace(',','.').astype(float)
df['emfn_ecor_ecri_diff'] = df['emfn_ecor_ecri_diff'].str.replace(',','.').astype(float)
df['emfn_ecor_emcn_diff'] = df['emfn_ecor_emcn_diff'].str.replace(',','.').astype(float)
df['emfn_emcn_epdn_diff'] = df['emfn_emcn_epdn_diff'].str.replace(',','.').astype(float)
df['epdn_ecne_ecpn_diff'] = df['epdn_ecne_ecpn_diff'].str.replace(',','.').astype(float)
df['epdn_eint_ecpn_diff'] = df['epdn_eint_ecpn_diff'].str.replace(',','.').astype(float)##
df['epdn_eint_espc_diff'] = df['epdn_eint_espc_diff'].str.replace(',','.').astype(float)
df['epdn_emfn_espc_diff'] = df['epdn_emfn_espc_diff'].str.replace(',','.').astype(float)
df['eplu_ecne_ecpn_diff'] = df['eplu_ecne_ecpn_diff'].str.replace(',','.').astype(float)
df['eplu_emeg_ecpn_diff'] = df['eplu_emeg_ecpn_diff'].str.replace(',','.').astype(float)
df['eplu_emeg_emal_diff'] = df['eplu_emeg_emal_diff'].str.replace(',','.').astype(float)
df['eplu_emsg_emal_diff'] = df['eplu_emsg_emal_diff'].str.replace(',','.').astype(float)


df = shuffle(df)

_dftarget = df['evento'].astype(int).values

format = '%d/%m/%Y'
df['date'] = pd.to_datetime(df['day'], format=format)
df = df.set_index('date')
df = df.drop(columns=['day'])
df = df.drop(columns=['evento'])

for i in reversed(range(len(df.columns))):
  if df.dtypes[df.columns[i]] == 'object':
    df = df.drop(df.columns[i], axis=1)

normalized_df = (df - df.mean()) / df.std()
df = normalized_df

#print(df.info())


In [3]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from mlxtend.plotting import plot_decision_regions
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
import matplotlib.pyplot as plt

In [4]:
from sklearn import datasets, decomposition
pca_feature_number = 16
pca = decomposition.PCA(n_components = pca_feature_number)
pca.fit(df)
new_df = pca.transform(df)

for i in range(pca_feature_number):
  df['f_' + str(i)] = new_df[:,i]

df = df.filter(like='f_')
#print(df.info())

In [5]:
y = _dftarget
split_at = 150
plottare = False

#max_f1_score = 0
#max_f1_score_who = ""
max_accuracy_score = 0
max_accuracy_score_who = ""

for i in range(pca_feature_number):
  for j in range(pca_feature_number):
    for a in range(pca_feature_number):
      for b in range(pca_feature_number):
        for c in range(pca_feature_number):
          for d in range(pca_feature_number):

            if i<j and j<a and a<b and b<c and c<d:
            #if i==12 and j==13:

              X = df.values[:, [i,j,a,b,c,d]]
              who = "feature " + str(i) + " - " + str(j) + " - " + str(a) + " - " + str(b) + " - " + str(c) + " - " + str(d)

              X_train = X[:split_at]
              X_test = X[split_at:]

              y_train = y[:split_at]
              y_test = y[split_at:]

              lr = LogisticRegressionCV(cv=5, random_state=0, class_weight='balanced') #scoring per cambiare metrica
              lr.fit(X_train, y_train)

              acc_s = lr.score(X_train, y_train)*100
              acc_s_test = lr.score(X_test, y_test)*100

              if max_accuracy_score < acc_s+acc_s_test:
                    max_accuracy_score = acc_s+acc_s_test
                    max_accuracy_score_who = who
                    print( who + ";" + "{:.1f}".format(acc_s) + ";" + "{:.1f}".format(acc_s_test) +";" + "{:.1f}".format(acc_s+acc_s_test) )

#print( max_f1_score_who )
#print( max_accuracy_score_who )
#print( conf_mat )



feature 0 - 1 - 2 - 3 - 4 - 5;56.0;62.8;118.8
feature 0 - 1 - 2 - 3 - 4 - 6;58.7;60.5;119.1
feature 0 - 1 - 2 - 3 - 4 - 7;55.3;65.1;120.4
feature 0 - 1 - 2 - 3 - 4 - 9;54.7;67.4;122.1
feature 0 - 1 - 2 - 3 - 4 - 12;64.0;60.5;124.5
feature 0 - 1 - 2 - 3 - 7 - 10;58.0;67.4;125.4
feature 0 - 1 - 2 - 3 - 7 - 12;63.3;65.1;128.4
feature 0 - 1 - 2 - 3 - 9 - 12;66.7;62.8;129.5
feature 0 - 1 - 2 - 3 - 12 - 15;64.7;69.8;134.4
feature 0 - 2 - 3 - 4 - 12 - 14;62.7;72.1;134.8
feature 0 - 2 - 3 - 11 - 12 - 14;62.0;74.4;136.4
feature 0 - 2 - 5 - 7 - 11 - 12;63.3;74.4;137.8
feature 2 - 3 - 6 - 7 - 12 - 14;64.0;74.4;138.4
